In [44]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/meetgoel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/meetgoel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [45]:
df = pd.read_csv("spotify_millsongdata.csv")

In [46]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [70]:
df.tail(5)

,artist,song,text
4995,Jennifer Lopez,Enough (Country Remix),thi is a fake song jennif never releas it ... ...
4996,UB40,Guilty,"well hello babi come sit down , make your self..."
4997,Rihanna,A Girl Like Me,some girl play the game they all walk and talk...
4998,Europe,Open Your Heart,day fill with joy and day fill with sorrow i d...
4999,John Prine,Lets Talk Dirty In Hawaiian,"well , i pack my bag and bought myself a ticke..."


In [48]:
df.shape

(57650, 4)

In [49]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [50]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [51]:
df.head(10)

,artist,song,text
0,Bread,Family Doctor,Went to my family doctor \r\nFor a pain insid...
1,Outkast,Stankonia,(intro:) \r\nWhat does love look like? \r\nL...
2,Stone Temple Pilots,Bi-Polar Bear,So I'm letting it go again \r\nI'm halfway fu...
3,Rod Stewart,Lady Luck,Lady Luck \r\nHere I am on time \r\nProppin'...
4,Nina Simone,Fine And Mellow,My man he don't love me he treats me oh so mea...
5,Who,Cousin Kevin,"We're on our own, cousin, \r\nAll alone, cous..."
6,Hillsong,Know You More,You hold it all at once \r\nThe earth and all...
7,Tori Amos,Flicker,"[Verse 1] \r\nFlicker, yes flicker \r\nThere..."
8,Yngwie Malmsteen,Stronghold,Alright \r\nAlright \r\nYou sacrificed \r\n...
9,David Bowie,Chilly Down,When the sun goes down (when the sun goes down...


In [52]:
df['text'][0]

"Went to my family doctor  \r\nFor a pain inside my head  \r\nTold him that since you left me  \r\nI was feelin' like just about dead  \r\nDoctor, what can you give me  \r\nAnd should I take one or two  \r\nCan you tell me just what it's gonna do  \r\nWill it save me from what I'm goin' through  \r\nWill it ease the pain  \r\nWill it ease the pain  \r\nWill it help to pacify my brain  \r\nHe told me not to worry  \r\nHe said to come back next week  \r\nHe asked me for a picture  \r\nOf the girl who made me lose my sleep  \r\nWell, all week long I suffered  \r\nDidn't know what to do  \r\nBut went back to him when the week was  \r\nThrough  \r\nAnd he gave me an artificial you  \r\nAnd it eased the pain  \r\nYes, it eased the pain  \r\nIt completely satisfied my brain.\r\n\r\n"

In [53]:
df.shape

(5000, 3)

Preprocessing

In [54]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [55]:
df.tail(5)

,artist,song,text
4995,Jennifer Lopez,Enough (Country Remix),this is a fake song \r jennifer never release...
4996,UB40,Guilty,"well hello baby \r come sit down, make your s..."
4997,Rihanna,A Girl Like Me,some girls play the game \r they all walk and...
4998,Europe,Open Your Heart,days filled with joy and days filled with sorr...
4999,John Prine,Lets Talk Dirty In Hawaiian,"well, i packed my bags and bought myself a tic..."


In [56]:
import nltk
nltk.download('punkt_tab')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/meetgoel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [57]:
stemmer = PorterStemmer()

In [58]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [59]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [62]:
similarity[0]

array([1.        , 0.03007192, 0.04625318, ..., 0.09325694, 0.02805021,
       0.01258634])

In [68]:
def recommendation(song_name):
    idx = df[df['song'] == song_name].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [76]:
recommendation('Lets Talk Dirty In Hawaiian')

['Up On The Downside',
 "I Can't Hold Out",
 'I Talk To Jesus (Demo)',
 'Gimme What You Got',
 'Talked Myself Right Into It',
 'Watching You Without Me',
 'A World Alone',
 'Fly Me To The Moon',
 'Time And A Word',
 'Shop Talk',
 'Everybody Wants To Feel Like You',
 'Find Your Self In You',
 'Dirty Life And Times',
 'E',
 'Talk To Me',
 'Talking Out Of Turn',
 'Let It Be',
 'Softly Over',
 'My World, My Love, My Life',
 'Only You']

In [74]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))